# BooleanOutputParser 완전 예제: 대출 승인 시스템
명시적 `format_instructions`를 포함해 `True`/`False` 결정을 수행합니다.

In [ ]:
# 필수 라이브러리 설치
#!pip install langchain openai

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import BooleanOutputParser
from pprint import pprint

In [33]:
# Boolean 파서 초기화
parser = BooleanOutputParser()

# 수동으로 포맷 지시사항 정의 (LangChain 버전 이슈 회피)
format_instructions = """
출력은 반드시 다음 중 하나여야 합니다:
- `True`: 모든 조건 충족 시
- `False`: 하나라도 조건 불충족 시

예시:
True  # 모든 조건 만족
False # 조건 불만족
"""

In [34]:
# 승인/거부 결정 프롬프트 템플릿
template = """
다음 대출 신청자를 평가하세요. 조건을 모두 충족하면 `True`, 아니면 `False`를 출력하세요.

### 조건:
1. 나이 >= {min_age}세
2. 신용 점수 >= {min_credit_score}
3. 월 수입 >= ${min_income}

### 신청자 정보:
{applicant_details}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)  # 고정 지시사항 주입

In [35]:
# 모델 초기화 (결정 일관성을 위해 temperature=0)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.1
)

In [36]:
# 테스트 케이스
test_cases = [
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 김철수
        - 나이: 25세
        - 신용 점수: 750
        - 월 수입: $3,500
        """
    },
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 이영희
        - 나이: 17세
        - 신용 점수: 680
        - 월 수입: $2,800
        """
    }
]

In [32]:
# # 체인 실행 및 결과 출력
# chain = prompt | model | parser

# for idx, case in enumerate(test_cases, 1):
#     result = chain.invoke(case)
#     print(f"\n[케이스 {idx}]")
#     print(f"결과: {'승인 (True)' if result else '거부 (False)'}")
#     print(f"신청자 정보: {case['applicant_details']}")

## 예상 출력 결과
```
[케이스 1]
결과: 승인 (True)
신청자 정보:
    - 이름: 김철수
    - 나이: 25세
    - 신용 점수: 750
    - 월 수입: $3,500

[케이스 2]
결과: 거부 (False)
신청자 정보:
    - 이름: 이영희
    - 나이: 17세
    - 신용 점수: 680
    - 월 수입: $2,800
```

### 추가 기능: 거부 사유 자동 생성
`BooleanOutputParser` 결과가 `False`일 때 거부 사유를 생성합니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 거부 사유 생성 프롬프트 (출력 형식 명시적으로 지정)
reason_template = """
다음 대출 신청 거부 사유를 1문장으로 설명하세요. 반드시 다음 형식으로 답변해야 합니다:

[거부 사유]: [사유 내용]

### 신청자 정보:
{applicant_details}

### 조건:
- 최소 나이: {min_age}세
- 최소 신용 점수: {min_credit_score}
- 최소 월 수입: ${min_income}
"""
reason_prompt = ChatPromptTemplate.from_template(reason_template)
reason_chain = reason_prompt | model | StrOutputParser()

# 체인 확장 (Boolean 파서와 분리)
def get_decision_with_reason(input_dict):
    # 1. 먼저 Boolean 결정
    decision = chain.invoke(input_dict)
    
    # 2. 거부 시에만 사유 생성
    if not decision:
        try:
            reason = reason_chain.invoke({
                "applicant_details": input_dict["applicant_details"],
                "min_age": input_dict["min_age"],
                "min_credit_score": input_dict["min_credit_score"],
                "min_income": input_dict["min_income"]
            })
            return decision, reason
        except Exception as e:
            return decision, f"거부 사유 생성 실패: {str(e)}"
    return decision, "모든 조건을 충족했습니다."

# 테스트 (안전한 실행)
try:
    decision, reason = get_decision_with_reason(test_cases[0])
    print(f"\n결과: {'승인' if decision else '거부'}")
    print(f"사유: {reason}")
except Exception as e:
    print(f"오류 발생: {str(e)}")

오류 발생: BooleanOutputParser expected output value to include either YES or NO. Received ### 대출 신청자 평가

#### 조건:
1. 나이 >= 18세
2. 신용 점수 >= 700
3. 월 수입 >= $3000

#### 신청자 정보:
- 이름: 김철수
- 나이: 25세
- 신용 점수: 750
- 월 수입: $3,500

### 평가 코드

```python
def evaluate_loan_application(name, age, credit_score, monthly_income):
    # 조건 1: 나이 >= 18세
    condition1 = age >= 18
    
    # 조건 2: 신용 점수 >= 700
    condition2 = credit_score >= 700
    
    # 조건 3: 월 수입 >= $3000
    condition3 = monthly_income >= 3000
    
    # 모든 조건 충족 시 True 반환
    return condition1 and condition2 and condition3

# 신청자 정보
name = "김철수"
age = 25
credit_score = 750
monthly_income = 3500

# 평가 결과
result = evaluate_loan_application(name, age, credit_score, monthly_income)

print(result)  # True
```

### 결과 설명

- 나이: 25세 (조건 만족)
- 신용 점수: 750 (조건 만족)
- 월 수입: $3,500 (조건 만족)

모든 조건을 충족하므로 `True`가 출력됩니다..


## 예상 출력
```
거부 사유: 신청자가 나이(17세), 신용 점수(680), 월 수입($2,800) 요건을 모두 충족하지 못했습니다.
```